In [1]:
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm
from pathlib import Path

In [2]:
def parse(text):
    df = pd.DataFrame({"raw": text.strip().splitlines()})
    pos = df.raw.str.extract("(-?\d+).*?(-?\d+).*?(-?\d+)", expand=True)
    pos = pos.astype(int).values
    vel = pos.copy() * 0
    return pos, vel

def step_gravity(pos, vel):
    next_vel = vel.copy()
    pairs = itertools.combinations(range(pos.shape[0]), 2)
    for planet1, planet2 in pairs:
        diff = np.clip(pos[planet2] - pos[planet1], a_min=-1, a_max=1)
        next_vel[planet1] += diff
        next_vel[planet2] -= diff
    return next_vel

def step_position(pos, vel):
    return pos + vel

def step(pos, vel):
    vel1 = step_gravity(pos, vel)
    pos1 = step_position(pos, vel1)
    return pos1, vel1

def at(pos, vel, n):
    for _ in range(n):
        pos, vel = step(pos, vel)
    return pos, vel

def energy(pos, vel):
    return np.dot(np.abs(pos).sum(axis=1), np.abs(vel).sum(axis=1))

### Part 1

In [3]:
pos, vel = parse("""<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>""")
assert energy(*at(pos, vel, 10)) == 179


pos, vel = parse("""<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>""")
assert energy(*at(pos, vel, 100)) == 1940

In [4]:
%%time
text = Path("input.txt").read_text()
pos0, vel0 = parse(text)
energy(*at(pos0, vel0, 1000))

CPU times: user 32.2 ms, sys: 3.9 ms, total: 36.1 ms
Wall time: 32.1 ms


6220

### Part 2

In [5]:
%%time
pos, vel = pos0.copy(), vel0.copy()
cycle_xyz = np.array([0, 0, 0])
i = 0
while True:
    pos, vel = step(pos, vel)
    i += 1
    needed = np.argwhere(cycle_xyz == 0).ravel()
    if needed.size == 0:
        break
    for axis in needed:
        same_pos = (pos[:, axis] == pos0[:, axis]).all()
        same_vel = (vel[:, axis] == vel0[:, axis]).all()
        if same_pos and same_vel:
            print(f"setting axis {'xyz'[axis]} to {i}")
            cycle_xyz[axis] = i

setting axis x to 113028
setting axis y to 167624
setting axis z to 231614
CPU times: user 12.5 s, sys: 537 ms, total: 13 s
Wall time: 12.2 s


In [6]:
np.lcm.reduce(cycle_xyz)

548525804273976